In [179]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

# Importar o ficheiro
df = pd.read_csv('C:\\Users\\guilh\\Downloads\\datasetC2.csv', sep=",", header=0)


df_final = df.copy()
# Armazenar mapeamentos de rótulos
label_mappings = {}

# Identificar as colunas com dados categóricos (strings)
categoricas = df.select_dtypes(include=['object']).columns

# Aplicar LabelEncoder às colunas categóricas
label_encoder = LabelEncoder()
for col in categoricas:
    # Mapear os rótulos antes da transformação
    unique_labels = df[col].unique()
    label_encoder.fit(unique_labels)
    label_mappings[col] = dict(zip(unique_labels, label_encoder.transform(unique_labels)))
    df[col] = label_encoder.transform(df[col])

X, y = df.drop("price", axis=1), df["price"]

# Inicializar o modelo Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=1234)

# Treinar o modelo com todos os dados
rf.fit(X, y)

# Fazer previsões no conjunto de dados original
previsoes = rf.predict(X)


# Criar um novo DataFrame chamado Previsoes
Previsoes = pd.DataFrame()

# Adicionar as colunas 'previsoes' e 'probabilidades' ao DataFrame Previsoes
Previsoes['Price Prevision'] = previsoes

# Concatenar os DataFrames df e Previsoes ao longo das colunas
df_regressao = pd.concat([df_final, Previsoes], axis=1)

# Exibir o DataFrame resultante
print(df_regressao)


              brand  model_year    milage      fuel_type  \
0             Other        2014   71000.0       Gasoline   
1             Other        2015   68575.0       Gasoline   
2             Other        2018   69641.0         Hybrid   
3             Other        2021   33500.0       Gasoline   
4             Other        2018   75000.0       Gasoline   
...             ...         ...       ...            ...   
2176          Other        2017  160000.0       Gasoline   
2177          Other        2023   15840.0       Gasoline   
2178          Other        2016   52105.0  E85 Flex Fuel   
2179  Mercedes-Benz        2014   76850.0       Gasoline   
2180          Other        2018   74150.0       Gasoline   

                                                 engine transmission ext_col  \
0         285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
1         270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
2     208.0HP 2.5L 4 Cylinder Engine Ga

In [180]:
import pandas as pd

# Substitua 'coluna_preco' pelo nome real da coluna de preços em seu conjunto de dados
df_regressao['Real Category'] = pd.qcut(df_regressao['price'], q=[i/50 for i in range(51)])

# Criar rótulos com base nos intervalos (menor valor - maior valor)
df_regressao['Real Category'] = df_regressao['Real Category'].apply(lambda x: f'[{x.left} - {x.right}]')

# Exibir o DataFrame resultante
print(df_regressao)


              brand  model_year    milage      fuel_type  \
0             Other        2014   71000.0       Gasoline   
1             Other        2015   68575.0       Gasoline   
2             Other        2018   69641.0         Hybrid   
3             Other        2021   33500.0       Gasoline   
4             Other        2018   75000.0       Gasoline   
...             ...         ...       ...            ...   
2176          Other        2017  160000.0       Gasoline   
2177          Other        2023   15840.0       Gasoline   
2178          Other        2016   52105.0  E85 Flex Fuel   
2179  Mercedes-Benz        2014   76850.0       Gasoline   
2180          Other        2018   74150.0       Gasoline   

                                                 engine transmission ext_col  \
0         285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
1         270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
2     208.0HP 2.5L 4 Cylinder Engine Ga

In [181]:
df_regressao['Real Category']

0       [21990.0 - 22999.0]
1       [24999.0 - 26050.0]
2       [22999.0 - 24000.0]
3       [32850.0 - 34000.0]
4       [37900.0 - 39000.0]
               ...         
2176      [7499.0 - 8500.0]
2177    [65997.0 - 70000.0]
2178    [19995.8 - 20999.0]
2179    [32850.0 - 34000.0]
2180    [44900.0 - 46319.4]
Name: Real Category, Length: 2181, dtype: category
Categories (50, object): ['[1999.999 - 5159.4]' < '[5159.4 - 6500.0]' < '[6500.0 - 7499.0]' < '[7499.0 - 8500.0]' ... '[65997.0 - 70000.0]' < '[70000.0 - 75500.0]' < '[75500.0 - 85000.0]' < '[85000.0 - 96500.0]']

In [182]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Assuming df_regressao is your original DataFrame

# Criar uma cópia do DataFrame original
df_classificacao = df_regressao.copy()

# Identificar as colunas com dados categóricos (strings)
categoricas = df_classificacao.select_dtypes(include=['object']).columns

# Aplicar LabelEncoder às colunas categóricas
label_encoder = LabelEncoder()
for col in categoricas:
    # Mapear os rótulos antes da transformação
    unique_labels = df_classificacao[col].unique()
    label_encoder.fit(unique_labels)
    label_mappings[col] = dict(zip(unique_labels, label_encoder.transform(unique_labels)))
    df_classificacao[col] = label_encoder.transform(df_classificacao[col])

X_classificacao, y_classificacao = df_classificacao.drop("Real Category", axis=1), df_classificacao["Real Category"]

# Inicializar o modelo DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(max_depth=100, random_state=1234)

# Treinar o modelo com todos os dados
dt_classifier.fit(X_classificacao, y_classificacao)

# Aplicar LabelEncoder às colunas categóricas no conjunto de dados de previsão
# Usar os mapeamentos previamente armazenados
for col in categoricas:
    df_classificacao[col] = df_classificacao[col].map(label_mappings[col])

# Fazer previsões no conjunto de dados original
previsoes_classificacao = dt_classifier.predict(X_classificacao)

probabilidade_acerto = dt_classifier.predict_proba(X_classificacao)

graus_confianca= [max(prob) - sorted (prob) [-2] for prob in probabilidade_acerto]

# Criar um novo DataFrame chamado Previsoes_classificacao
Previsoes_classificacao = pd.DataFrame()

trust_rating = pd.DataFrame()

# Adicionar as colunas 'previsoes' ao DataFrame Previsoes_classificacao
Previsoes_classificacao['Category Prevision'] = previsoes_classificacao
trust_rating['Trust Rating (Classification)'] = graus_confianca; 

# Concatenar os DataFrames df_classificacao e Previsoes_classificacao ao longo das colunas
df_resultado_classificacao = pd.concat([df_regressao, Previsoes_classificacao, trust_rating], axis=1)

print(df_resultado_classificacao)

              brand  model_year    milage      fuel_type  \
0             Other        2014   71000.0       Gasoline   
1             Other        2015   68575.0       Gasoline   
2             Other        2018   69641.0         Hybrid   
3             Other        2021   33500.0       Gasoline   
4             Other        2018   75000.0       Gasoline   
...             ...         ...       ...            ...   
2176          Other        2017  160000.0       Gasoline   
2177          Other        2023   15840.0       Gasoline   
2178          Other        2016   52105.0  E85 Flex Fuel   
2179  Mercedes-Benz        2014   76850.0       Gasoline   
2180          Other        2018   74150.0       Gasoline   

                                                 engine transmission ext_col  \
0         285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
1         270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
2     208.0HP 2.5L 4 Cylinder Engine Ga

In [183]:
df_resultado_classificacao['Price Prevision'] = df_resultado_classificacao['Price Prevision'].astype(int)

df_resultado_classificacao['id'] = range(1, len(df_resultado_classificacao) + 1)

print(df_resultado_classificacao['Price Prevision'])

0       20943
1       23687
2       22350
3       34359
4       36432
        ...  
2176     9563
2177    63724
2178    20545
2179    31616
2180    43824
Name: Price Prevision, Length: 2181, dtype: int32


In [184]:
df_resultado_classificacao = df_resultado_classificacao[['id'] + [col for col in df_resultado_classificacao.columns if col != 'id']]
df_resultado_classificacao['price_difference'] = abs(df_resultado_classificacao['price'] - df_resultado_classificacao['Price Prevision'])
df_resultado_classificacao['error_percentage'] = abs(df_resultado_classificacao['price_difference'] / df_resultado_classificacao['price'] * 100).round(2)
df_resultado_classificacao['categoria_acertividade'] = df_resultado_classificacao['Real Category'] == df_resultado_classificacao['Category Prevision']


In [185]:
df_resultado_classificacao['price']

0       22000
1       26000
2       23995
3       34000
4       38095
        ...  
2176     7750
2177    69900
2178    20499
2179    32999
2180    45000
Name: price, Length: 2181, dtype: int64

In [186]:
print(df_resultado_classificacao["price"], df_resultado_classificacao["Price Prevision"], df_resultado_classificacao['price_difference'], df_resultado_classificacao["error_percentage"])

0       22000
1       26000
2       23995
3       34000
4       38095
        ...  
2176     7750
2177    69900
2178    20499
2179    32999
2180    45000
Name: price, Length: 2181, dtype: int64 0       20943
1       23687
2       22350
3       34359
4       36432
        ...  
2176     9563
2177    63724
2178    20545
2179    31616
2180    43824
Name: Price Prevision, Length: 2181, dtype: int32 0       1057
1       2313
2       1645
3        359
4       1663
        ... 
2176    1813
2177    6176
2178      46
2179    1383
2180    1176
Name: price_difference, Length: 2181, dtype: int64 0        4.80
1        8.90
2        6.86
3        1.06
4        4.37
        ...  
2176    23.39
2177     8.84
2178     0.22
2179     4.19
2180     2.61
Name: error_percentage, Length: 2181, dtype: float64


In [187]:
# Loop through each column and replace '.' with ','
for col in df_resultado_classificacao.columns:
    df_resultado_classificacao[col] = df_resultado_classificacao[col].astype(str).str.replace('.', ',')

# Display the modified DataFrame
print(df_resultado_classificacao)


# If you want to replace only in specific columns, you can do the following:
# df_resultado_classificacao[['column1', 'column2', ...]] =


        id          brand model_year    milage      fuel_type  \
0        1          Other       2014   71000,0       Gasoline   
1        2          Other       2015   68575,0       Gasoline   
2        3          Other       2018   69641,0         Hybrid   
3        4          Other       2021   33500,0       Gasoline   
4        5          Other       2018   75000,0       Gasoline   
...    ...            ...        ...       ...            ...   
2176  2177          Other       2017  160000,0       Gasoline   
2177  2178          Other       2023   15840,0       Gasoline   
2178  2179          Other       2016   52105,0  E85 Flex Fuel   
2179  2180  Mercedes-Benz       2014   76850,0       Gasoline   
2180  2181          Other       2018   74150,0       Gasoline   

                                                 engine transmission ext_col  \
0         285,0HP 3,6L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
1         270,0HP 3,5L V6 Cylinder Engine Gasoline Fuel    

In [188]:
import random
from datetime import datetime, timedelta
import pandas as pd

# Adicionando coluna de data
data_inicio = datetime(2023, 12, 1)
data_fim = datetime(2024, 1, 31)
dias = len(df_resultado_classificacao)

# Gerando datas aleatórias para cada linha
datas_aleatorias = [data_inicio + timedelta(days=random.randint(0, (data_fim - data_inicio).days)) for _ in range(dias)]
df_resultado_classificacao['data'] = datas_aleatorias

# Adicionando coluna de número de utilizadores
num_utilizadores = []
for data in df_resultado_classificacao['data']:
    if data.month == 12:  # Para o primeiro mês (dezembro)
        num_utilizadores.append(random.randint(50, 500))
    elif data.month == 1:  # Para o segundo mês (janeiro)
        num_utilizadores.append(random.randint(500, 1000))

df_resultado_classificacao['num_utilizadores'] = num_utilizadores

# Garantindo que o número de utilizadores seja o mesmo para a mesma data
df_resultado_classificacao['num_utilizadores'] = df_resultado_classificacao.groupby('data')['num_utilizadores'].transform('first')

# Exibindo o DataFrame resultante
print(df_resultado_classificacao)


        id          brand model_year    milage      fuel_type  \
0        1          Other       2014   71000,0       Gasoline   
1        2          Other       2015   68575,0       Gasoline   
2        3          Other       2018   69641,0         Hybrid   
3        4          Other       2021   33500,0       Gasoline   
4        5          Other       2018   75000,0       Gasoline   
...    ...            ...        ...       ...            ...   
2176  2177          Other       2017  160000,0       Gasoline   
2177  2178          Other       2023   15840,0       Gasoline   
2178  2179          Other       2016   52105,0  E85 Flex Fuel   
2179  2180  Mercedes-Benz       2014   76850,0       Gasoline   
2180  2181          Other       2018   74150,0       Gasoline   

                                                 engine transmission ext_col  \
0         285,0HP 3,6L V6 Cylinder Engine Gasoline Fuel    Automatic   Other   
1         270,0HP 3,5L V6 Cylinder Engine Gasoline Fuel    

In [189]:
df_resultado_classificacao

,id,brand,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,cylinders,Price Prevision,Real Category,Category Prevision,Trust Rating (Classification),price_difference,error_percentage,categoria_acertividade,data,num_utilizadores
0,1,Other,2014,"71000,0",Gasoline,"285,0HP 3,6L V6 Cylinder Engine Gasoline Fuel",Automatic,Other,Black,None reported,...,"6,0",20943,"[21990,0 - 22999,0]","[21990,0 - 22999,0]","1,0",1057,"4,8",True,2024-01-01,734
1,2,Other,2015,"68575,0",Gasoline,"270,0HP 3,5L V6 Cylinder Engine Gasoline Fuel",Automatic,Other,Gray,At least 1 accident or damage reported,...,"6,0",23687,"[24999,0 - 26050,0]","[24999,0 - 26050,0]","1,0",2313,"8,9",True,2023-12-09,472
2,3,Other,2018,"69641,0",Hybrid,"208,0HP 2,5L 4 Cylinder Engine Gas/Electric Hy...",Automatic,White,Black,At least 1 accident or damage reported,...,"4,0",22350,"[22999,0 - 24000,0]","[22999,0 - 24000,0]","1,0",1645,"6,86",True,2023-12-13,450
3,4,Other,2021,"33500,0",Gasoline,"260,0HP 2,4L 4 Cylinder Engine Gasoline Fuel",Automatic,Other,Other,None reported,...,"4,0",34359,"[32850,0 - 34000,0]","[32850,0 - 34000,0]","1,0",359,"1,06",True,2024-01-22,597
4,5,Other,2018,"75000,0",Gasoline,"301,0HP 4,6L 8 Cylinder Engine Gasoline Fuel",Automatic,White,Beige,At least 1 accident or damage reported,...,"8,0",36432,"[37900,0 - 39000,0]","[37900,0 - 39000,0]","1,0",1663,"4,37",True,2023-12-25,416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176,2177,Other,2017,"160000,0",Gasoline,"152,0HP 2,0L 4 Cylinder Engine Gasoline Fuel",Automatic,White,Beige,At least 1 accident or damage reported,...,"4,0",9563,"[7499,0 - 8500,0]","[7499,0 - 8500,0]","1,0",1813,"23,39",True,2023-12-08,346
2177,2178,Other,2023,"15840,0",Gasoline,"296,0HP 2,0L 4 Cylinder Engine Gasoline Fuel",Automatic,White,Black,None reported,...,"4,0",63724,"[65997,0 - 70000,0]","[65997,0 - 70000,0]","1,0",6176,"8,84",True,2024-01-29,880
2178,2179,Other,2016,"52105,0",E85 Flex Fuel,"220,0HP 2,0L 4 Cylinder Engine Flex Fuel Capab...",Automatic,White,Beige,At least 1 accident or damage reported,...,"4,0",20545,"[19995,8 - 20999,0]","[19995,8 - 20999,0]","1,0",46,"0,22",True,2024-01-24,841
2179,2180,Mercedes-Benz,2014,"76850,0",Gasoline,"429,0HP 4,7L 8 Cylinder Engine Gasoline Fuel",Automatic,Other,Beige,At least 1 accident or damage reported,...,"8,0",31616,"[32850,0 - 34000,0]","[32850,0 - 34000,0]","1,0",1383,"4,19",True,2023-12-26,124


In [190]:
df_resultado_classificacao.to_csv('Previsões.csv', index=False)